# import packages


In [ ]:
import numpy as np
import pandas as pd
import glob,os
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import random
#use to make all features into one sentence
from sklearn.model_selection import train_test_split
#For resampling
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler,ADASYN,SMOTE
from imblearn.combine import SMOTEENN,SMOTETomek
from collections import Counter

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
colab=1 # set it top 0 if you need jupyter

if colab:
  from google.colab import drive
  drive.mount('/content/drive',force_remount=True)
  working_dir='/content/drive/My Drive/ponzi/'
  results_dir = '/content/drive/My Drive/ponzi/'
  os.chdir(working_dir)

Mounted at /content/drive


# functions

In [ ]:
minmaxscaler = MinMaxScaler(feature_range=(0, 1))
standardscaler = StandardScaler()

In [ ]:
def get_traintest(df,labelname):
  df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
  train, test = train_test_split(df, test_size = 0.2)
  train_x = train.drop([labelname],axis=1)
  train_y = train[labelname]
  test_x = test.drop([labelname],axis=1)
  test_y = test[labelname]
  train_x = standardscaler.fit_transform(train_x)
  test_x = standardscaler.fit_transform(test_x)
  return train_x,train_y,test_x,test_y,train,test

In [ ]:
def etc(train_x,train_y,test_x,test_y):
  #param_grid = { }
  ETC_CV = ExtraTreesClassifier()
  #ETC_CV = GridSearchCV(estimator=ExtraTreesClassifier(), param_grid=param_grid, cv= 5,scoring="f1")
  ETC_CV.fit(train_x, train_y)
  #print("Best parameters: ",ETC_CV.best_params_)
  #history = ETC_CV.best_estimator_.fit(train_x, train_y)
  y_pred = ETC_CV.predict(test_x) 
  test_y = np.array(test_y)
  print(classification_report(test_y, y_pred))
  ans = compute_measure_multiclass(test_y, y_pred)
  return y_pred,ans

In [ ]:
def doResamapling(data, label, method='RandomOverSampler'):
    if method=='SMOTE':
        resampling_obj=SMOTE(random_state=42)
    elif method=='RandomOverSampler':
        resampling_obj=RandomOverSampler(random_state=42)
    elif method=='SMOTEENN':
        resampling_obj=SMOTEENN(random_state=42)
    elif method=='ADASYN':
        resampling_obj=ADASYN(random_state=42)
    elif method=='TomekLinks':
    #resampling_obj=TomekLinks(random_state=42, sampling_strategy='auto')
        resampling_obj=TomekLinks(sampling_strategy='auto')
    else:
        resampling_obj=SMOTETomek(random_state=42)

    data_res, label_res=resampling_obj=resampling_obj.fit_resample(data, label)
    return data_res, label_res

In [ ]:
metrics = ['Accuracy','sensitivity','specificity','positive prediction ratio','negative prediction ratio','F1 score','d-index']

# data

In [ ]:
data = pd.read_csv('Ponzi.csv')
data.head()

,Unnamed: 0,address,lifetime,most active,active day,in vs out,num out,num in,num addr received,num addr sent,median delay,mean delay,minimum delay,maximum delay,total received BTC,total sent BTC,avg received BTC,avg sent BTC,48 diff,class
0,1,114Ap9G5nu78vESC648amPwSeqUorPtV5L,28329132,202,1035,1.004892,1022,1027,3430,3127,4369,27397.61315,-64,2881479,9.737518,9.746409,0.009528,0.009490,-200,1
1,3,1168Hc7aRi1azJPFRbCMGFvS1Q7R4taiZ3,1952932,67,14,0.126126,7,7,14,111,4559,150225.53850,473,1031864,0.084129,0.084129,0.012018,0.012018,-65,1
2,4,121iUY1czgXaQcEFrKKjbzMUdCquTQgPHL,17622640,71,165,1.000000,86,86,1625,360,25496,107455.12200,0,989807,76.400340,76.400340,0.888376,0.888376,-69,1
3,5,121jfy7iR2Q6HCNL3oZFDK2WTjigFZRAVb,103352586,26,348,1.000000,208,208,752,672,362,297846.06920,-427,102562674,44.303918,44.303918,0.213000,0.213000,22,1
4,6,121x1rbwtPk4jg48V42zv1CprdY5Lm1ozb,3172652,14,82,1.000000,53,53,201,148,0,39168.54321,-839,2754080,12.938357,12.938357,0.244120,0.244120,12,1


In [ ]:
data = data.drop(['Unnamed: 0','address'],axis=1)
print('NA:',data.isnull().sum().max())
data.describe()

NA: 0


,lifetime,most active,active day,in vs out,num out,num in,num addr received,num addr sent,median delay,mean delay,minimum delay,maximum delay,total received BTC,total sent BTC,avg received BTC,avg sent BTC,48 diff,class
count,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6.819000e+03,6.819000e+03,6.819000e+03,6.819000e+03,6.819000e+03,6.819000e+03,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000,6819.000000
mean,2.694412e+07,193.174512,187.487315,3.586249,158.757002,151.812876,2.923279e+03,3.444426e+03,8.182274e+05,1.347796e+06,5.727570e+05,1.271390e+07,753.319665,696.568270,7.030351,6.822009,-39.519871,1.075964
std,5.254176e+07,385.418165,1753.027220,18.562206,1573.447201,1518.284236,3.350098e+04,3.430694e+04,4.078997e+06,4.278853e+06,4.392918e+06,3.058855e+07,11860.103898,10263.775252,103.872685,101.717595,1242.214401,0.975999
min,0.000000e+00,2.000000,2.000000,0.000828,0.000000,0.000000,2.000000e+00,2.000000e+00,-9.000000e+00,0.000000e+00,-1.596362e+08,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-50055.000000,0.000000
25%,6.229265e+05,16.000000,4.000000,0.412544,2.000000,2.000000,1.600000e+01,2.200000e+01,1.083350e+04,4.867371e+04,0.000000e+00,2.782045e+05,0.073466,0.078521,0.015000,0.015344,-96.000000,0.000000
50%,5.737272e+06,81.000000,27.000000,1.000000,17.000000,19.000000,9.800000e+01,1.240000e+02,4.669800e+04,1.774707e+05,3.260000e+02,1.491815e+06,0.985315,1.049264,0.081420,0.083709,0.000000,2.000000
75%,2.629540e+07,177.000000,76.000000,1.142857,51.000000,54.000000,3.610000e+02,1.153000e+03,1.367935e+05,7.216448e+05,7.485000e+03,9.410970e+06,28.927453,31.110504,0.796665,0.812092,26.000000,2.000000
max,3.343655e+08,4856.000000,82800.000000,709.237881,63150.000000,74719.000000,1.767354e+06,1.998305e+06,1.109263e+08,1.109263e+08,1.109263e+08,3.023198e+08,623015.144500,559547.264200,4356.869814,4356.869814,78643.000000,2.000000


In [ ]:
labelname = 'class'

In [ ]:
print("bitcoin data set dimensions : {}".format(data.shape))

bitcoin data set dimensions : (6819, 18)


In [ ]:
data.groupby(labelname).size()

class
0    3008
1     285
2    3526
dtype: int64

In [ ]:
train_x,train_y,test_x,test_y,train,test = get_traintest(data,labelname)

In [ ]:
train_entropy = computeDataEntropy(train_x)
print('train_entropy',train_entropy)
test_entropy = computeDataEntropy(test_x)
print('test_entropy',test_entropy)

train_entropy 12.627252184194003
test_entropy 11.358618163583836


In [ ]:
method=['RandomOverSampler', 'SMOTE', 'SMOTEENN', 'ADASYN', 'SMOTETomek','TomekLinks']
train_set = []
for e in method:
  train_label = train[labelname]
  train0 = train.drop([labelname],axis=1)
  print(e)
  print('-------------------------------------')
  data_res, label_res=doResamapling(train0, train_label, e)
  t=Counter(label_res)
  major_ratio=t[0]/(t[0]+t[1]+t[2])
  print('The majority type ratio {:.2f}%'.format(major_ratio*100))
  print(t[0], t[1],t[2])
  entropy=computeDataEntropy(data_res)
  print('The train data entropy after'+e+ '{:3.4f}'.format(entropy))
  res_df = pd.DataFrame(data = data_res, columns=train0.columns)
  res_df[labelname]=label_res  
  train_set.append(res_df)

RandomOverSampler
-------------------------------------
The majority type ratio 33.33%
2832 2832 2832
The train data entropy afterRandomOverSampler38.0025
SMOTE
-------------------------------------
The majority type ratio 33.33%
2832 2832 2832
The train data entropy afterSMOTE37.9687
SMOTEENN
-------------------------------------
The majority type ratio 28.07%
1055 1671 1032
The train data entropy afterSMOTEENN37.5935
ADASYN
-------------------------------------
The majority type ratio 31.25%
2560 2799 2832
The train data entropy afterADASYN37.8961
SMOTETomek
-------------------------------------
The majority type ratio 32.42%
2373 2563 2384
The train data entropy afterSMOTETomek37.9146
TomekLinks
-------------------------------------
The majority type ratio 42.61%
1959 222 2416
The train data entropy afterTomekLinks37.5745


In [ ]:
yhat,ans = etc(train_x,train_y,test_x,test_y)

              precision    recall  f1-score   support

           0       0.90      0.52      0.66       607
           1       0.42      0.13      0.20        63
           2       0.67      0.96      0.79       694

    accuracy                           0.73      1364
   macro avg       0.66      0.54      0.55      1364
weighted avg       0.76      0.73      0.71      1364

diagnostic index: 1.559786427851868
F1 score : 0.7842227378190255
Accuracy: 0.7272727272727273
sensitivity: 0.892998678996037
specificity: 0.5205930807248764
negative prediction ratio: 0.7959697732997482
positive prediction ratio: 0.6990692864529473
[0.7272727272727273, 0.892998678996037, 0.5205930807248764, 0.6990692864529473, 0.7959697732997482, 0.7842227378190255, 1.559786427851868]


In [ ]:
result_ET = []
result_ET.append(ans)
np.save(results_dir+'_ponzi_ET',np.array(ans))
results_df = pd.DataFrame(data=ans, index=metrics, columns=['metric'])

resampling

In [ ]:
test = test.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
for i in range(len(method)):
  print(method[i])
  print('-----------------------------------------------')
  train_set[i] = train_set[i].replace([np.inf, -np.inf], np.nan).dropna(axis=0)
  train_x,train_y,test_x,test_y = get_newtraintest(train_set[i],test)
  yhat,ans = etc(train_x,train_y,test_x,test_y)
  #result_cnn.append(ansnew)
  np.save(results_dir+'_ponzi_ET'+ method[i],np.array(ans))
  print('-----------------------------------------------')

RandomOverSampler
-----------------------------------------------
              precision    recall  f1-score   support

           0       0.91      0.46      0.61       607
           1       0.39      0.14      0.21        63
           2       0.64      0.95      0.77       694

    accuracy                           0.70      1364
   macro avg       0.65      0.52      0.53      1364
weighted avg       0.75      0.70      0.67      1364

diagnostic index: 1.5076624627542312
F1 score : 0.7651083238312428
Accuracy: 0.6979472140762464
sensitivity: 0.8863936591809776
specificity: 0.46293245469522243
negative prediction ratio: 0.7656675749318801
positive prediction ratio: 0.6730190571715146
[0.6979472140762464, 0.8863936591809776, 0.46293245469522243, 0.6730190571715146, 0.7656675749318801, 0.7651083238312428, 1.5076624627542312]
-----------------------------------------------
SMOTE
-----------------------------------------------
              precision    recall  f1-score   support

 